In [1]:
!pip install transformers gradio datasets > /dev/null

In [ ]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

In [3]:
def user(message, history):
    return "", history + [[message, None]]

In [4]:
def bot(history):

    user_message = history[-1][0]

    new_user_input_ids = tokenizer.encode(
        user_message + tokenizer.eos_token,
        return_tensors="pt"
    )

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([torch.LongTensor([]),
                               new_user_input_ids], dim=-1)

    # generate a response
    response = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id
    ).tolist()

    # convert the tokens to text, and then split the responses into lines
    response = tokenizer.decode(response[0]).split("<|endoftext|>")

    response = [(response[i],
                 response[i + 1]) for i in range(0,
                                                 len(response) - 1,
                                                 2)]  # convert to tuples of list
    history[-1] = response[0]

    return history


In [28]:
user_message = "This is a nice cool breezy weather"

new_user_input_ids = tokenizer.encode(
        user_message + tokenizer.eos_token,
        return_tensors="pt"
    )

In [29]:
# append the new user input tokens to the chat history
bot_input_ids = torch.cat([torch.LongTensor([]),
                               new_user_input_ids], dim=-1)

In [30]:
# generate a response
response = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id
    ).tolist()

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [32]:
response[0]

[1212,
 318,
 257,
 3621,
 3608,
 20780,
 7357,
 6193,
 50256,
 1026,
 338,
 257,
 3621,
 1110,
 329,
 257,
 20780,
 7357,
 1110,
 50256]

There are three Gradio components here:

    A Chatbot, whose value stores the entire history of the conversation, as a list of response pairs between the user and bot.

    A Textbox where the user can type their message, and then hit enter/submit to trigger the chatbot response

    A ClearButton button to clear the Textbox and entire Chatbot history


In [ ]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()


In [25]:
def multiplier(message, history):

    iputs = message.split('x')

    # append the new user input tokens to the chat history
    oput = str(int(iputs[0]) * int(iputs[1]))

    history.append((message,oput))

    return "", history


In [27]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    msg.submit(multiplier, [msg,
                      chatbot], [msg,
                                 chatbot])

demo.launch(debug = True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
